<a href="https://colab.research.google.com/github/BrunoFerCam/BotYugioh/blob/main/botPeixaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pyTelegramBotAPI transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 1.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import telebot
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model_name = 'microsoft/DialoGPT-medium'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
API_TOKEN = '7469665499:AAHPUUNbXgmh6bJXPKgXb5asplhCfaJA9kM'
bot = telebot.TeleBot(API_TOKEN)

In [ ]:
knowledge_base = {
    "store_hours": "Our store is open from 9 AM to 7 PM, Monday to Saturday. On Sundays, we are open from 10 AM to 4 PM.",
    "discounts": "We currently have a 15% discount on all fish tanks over 50 gallons.",
    "product_prices": {
        "20_gallon_tank": "$50",
        "50_gallon_tank": "$100",
        "filter": "$20"
    }
}

In [ ]:
def fetch_knowledge_base_response(user_input):
    greetings = ["hello", "hi", "hey"]
    if any(greeting in user_input.lower() for greeting in greetings):
        return "Hello! Welcome to Lambari Aquarium. How can I assist you today?"
    elif "store hours" in user_input.lower():
        return knowledge_base["store_hours"]
    elif "discounts" in user_input.lower():
        return knowledge_base["discounts"]
    elif "price" in user_input.lower():
        if "20 gallon tank" in user_input.lower():
            return f"The price of a 20-gallon tank is {knowledge_base['product_prices']['20_gallon_tank']}."
        elif "50 gallon tank" in user_input.lower():
            return f"The price of a 50-gallon tank is {knowledge_base['product_prices']['50_gallon_tank']}."
        elif "filter" in user_input.lower():
            return f"The price of a filter is {knowledge_base['product_prices']['filter']}."
    return None


@bot.message_handler(func=lambda message: True)
def handle_message(message):
    user_input = message.text

    kb_response = fetch_knowledge_base_response(user_input)
    if kb_response:
        bot.reply_to(message, kb_response)
    else:
        inputs = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
        outputs = model.generate(inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)

        response = tokenizer.decode(outputs[:, inputs.shape[-1]:][0], skip_special_tokens=True)
        bot.reply_to(message, response)

In [ ]:
bot.polling()